In [1]:
import os
from pathlib import Path
import glob
import numpy as np
import pandas as pd
import geopandas as gpd
from datetime import datetime
import matplotlib.pyplot as plt
import geoutils as gu
import xdem
from pprint import pprint
import altair as alt    
from rasterio.enums import Resampling
import json 
import seaborn as sns
from shapely import wkt
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

from functools import reduce
import scipy

np.set_printoptions(suppress=True)
from sklearn.metrics import r2_score

In [2]:
def nse(targets,predictions):
    return 1-(np.sum((targets-predictions)**2)/np.sum((targets-np.mean(predictions))**2))

In [3]:
def r2_plot(df, x, y, plot_fit_line=False, r2_annotations=False, limit=150000, x_axis_title='', y_axis_title='', size=100):
    chart = alt.Chart(df).mark_circle(size=size).encode(
        x=alt.X(x, title=x_axis_title),
        y=alt.Y(y, title=y_axis_title)
    )
    line = chart.transform_regression(x, y).mark_line()

    params = alt.Chart(df).transform_regression(
        x, y, params=True
    ).mark_text(align='left').encode(
        x=alt.value(20),  # pixels from left
        y=alt.value(20),  # pixels from top
        text=alt.Text('rSquared:N', format=".3f")
    ).properties(width=200, height=200)

    one_to_one_line = alt.Chart(pd.DataFrame({
        'x': np.linspace(0, limit, 100),
        'y': np.linspace(0, limit, 100)
    })).mark_line(color='black', opacity=0.25).encode(
        alt.X('x', scale=alt.Scale(domain=(0,limit))),
        alt.Y('y', scale=alt.Scale(domain=(0,limit)))
    )

    if r2_annotations:
        base_chart = (chart + params + one_to_one_line).properties(width=200, height=200)
    else:
        base_chart = (chart + one_to_one_line).properties(width=200, height=200)
    if plot_fit_line:
        return base_chart + line
    else:
        return base_chart

# Set constants

In [4]:
porosity = 0.35
density_kg_per_cubic_meter = 2600
kg_per_metric_ton = 1000

# Load Data

### Load streamstats wastersheds

In [5]:
streamstats_watersheds_fns = glob.glob("/data2/elilouis/hsfm-geomorph/data/mt_baker_mass_wasted/streamstats_watersheds/*.geojson")

streamstats_gdf = gpd.GeoDataFrame()
for f in streamstats_watersheds_fns:
    new_data = gpd.read_file(f)
    new_data['Valley Name'] = f.split("/")[-1].split(".geojson")[0]
    streamstats_gdf = pd.concat([streamstats_gdf, new_data])
streamstats_gdf['Valley Name'] = streamstats_gdf['Valley Name'].str.title()
streamstats_gdf = streamstats_gdf[streamstats_gdf.geometry.type == 'Polygon']
streamstats_gdf = streamstats_gdf.to_crs("EPSG:32610")
streamstats_gdf['watershed area (square m)'] = streamstats_gdf.geometry.area
streamstats_gdf['watershed area (square km)'] = streamstats_gdf['watershed area (square m)'] / 10**6

In [6]:
streamstats_gdf

,id,geometry,Valley Name,watershed area (square m),watershed area (square km)
1,globalwatershed,"POLYGON ((585832.641 5402803.324, 585718.910 5...",Coleman,1.385497e+07,13.854967
1,globalwatershed,"POLYGON ((587209.419 5404136.340, 587180.988 5...",Mazama,1.319677e+07,13.196766
1,globalwatershed,"POLYGON ((588484.771 5404964.334, 588476.361 5...",Rainbow,1.285606e+07,12.856056
1,globalwatershed,"POLYGON ((591315.851 5401801.442, 591258.986 5...",Park,9.948008e+06,9.948008
1,globalwatershed,"POLYGON ((586708.849 5403194.338, 586936.305 5...",Boulder,5.634194e+06,5.634194
1,globalwatershed,"POLYGON ((588799.807 5398100.063, 588771.370 5...",Squak,3.947675e+06,3.947675
1,globalwatershed,"POLYGON ((586136.546 5396189.492, 586108.105 5...",Easton,4.698693e+06,4.698693
1,globalwatershed,"POLYGON ((582622.520 5398292.189, 582508.765 5...",Deming,1.158924e+07,11.589236
1,globalwatershed,"POLYGON ((589897.579 5399222.172, 589840.707 5...",Talum,3.130098e+06,3.130098
1,globalwatershed,"POLYGON ((582975.834 5400707.645, 582862.090 5...",Thunder,7.110903e+06,7.110903


### Load glacier change measurements

In [7]:
glacier_change_df = pd.read_pickle("outputs/glacier_area.pickle").reset_index()

In [8]:
glacier_change_df = pd.read_pickle("outputs/glacier_area.pickle").reset_index()
glacier_change_df['Valley Name'] = glacier_change_df['Name'].apply(lambda s: s.split(" ")[0])
glacier_change_df['glacial advance absolute'] = glacier_change_df.apply(
    lambda row: row['area difference']['1977_09_27'] 
        if not np.isnan(row['area difference']['1977_09_27'])
        else row['area difference']['1979_10_06'],
    axis=1
)
glacier_change_df['glacial retreat absolute'] = glacier_change_df['area difference']['2015_09_01']
glacier_change_df['glacial advance and retreat absolute'] = np.abs(glacier_change_df['glacial advance absolute']) + np.abs(glacier_change_df['glacial retreat absolute'])
glacier_change_df['glacial area 1947'] = glacier_change_df['area']['1947_09_14']
glacier_change_df['glacial area 1977/79'] = glacier_change_df['area'].apply(lambda row: row['1977_09_27'] if np.isnan(row['1979_10_06']) else row['1979_10_06'], axis=1)
glacier_change_df['glacial area 2015'] = glacier_change_df['area']['2015_09_01']

glacier_change_df = glacier_change_df[[
    'Valley Name',
    'glacial advance absolute',
    'glacial retreat absolute',
    'glacial advance and retreat absolute',
    'glacial area 1947',
    'glacial area 1977/79',
    'glacial area 2015'
]]

glacier_change_df.columns = glacier_change_df.columns.get_level_values(0)

# Manually combine the Coleman Roosevelt rows, remove the old ones, and add a new combined "Coleman" row
row = glacier_change_df[glacier_change_df['Valley Name'].isin(['Coleman', 'Roosevelt'])].sum()
row['Valley Name'] = 'Coleman'
glacier_change_df = glacier_change_df[~glacier_change_df['Valley Name'].isin(['Coleman', 'Roosevelt'])].append(row, ignore_index=True)

/tmp/ipykernel_106507/2584832076.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  glacier_change_df = glacier_change_df[~glacier_change_df['Valley Name'].isin(['Coleman', 'Roosevelt'])].append(row, ignore_index=True)


### Load terrain attribute data

In [9]:
terrain_attrs_erosionarea = pd.read_csv("outputs/terrain_attributes_erosionarea.csv")
terrain_attrs_erosionarea = terrain_attrs_erosionarea.rename(columns={'name': 'Valley Name'})
terrain_attrs_erosionarea['drainage area (km)'] = terrain_attrs_erosionarea['drainage area'] / 1e6


terrain_attrs_erosionarea_bytpe = pd.read_csv("outputs/terrain_attributes_erosionarea_bytype.csv")
terrain_attrs_erosionarea_bytpe = terrain_attrs_erosionarea_bytpe.rename(columns={'name': 'Valley Name'})
terrain_attrs_erosionarea_bytpe['drainage area (km)'] = terrain_attrs_erosionarea_bytpe['drainage area'] / 1e6

In [10]:
terrain_attrs_erosionarea[['slope all erosion area', 'drainage area (km)', 'curvature']] = terrain_attrs_erosionarea[['slope', 'drainage area (km)', 'curvature']]

terrain_attrs_erosionarea = terrain_attrs_erosionarea[['Valley Name', 'drainage area (km)', 'curvature', 'slope all erosion area']]

In [11]:
terrain_attrs_erosionarea = terrain_attrs_erosionarea.merge(
    terrain_attrs_erosionarea_bytpe.query("type == 'fluvial'")[['Valley Name', 'slope']].rename(columns={'slope': 'slope fluvial erosion area'}),
    on='Valley Name'
)

terrain_attrs_erosionarea = terrain_attrs_erosionarea.merge(
    terrain_attrs_erosionarea_bytpe.query("type == 'hillslope'")[['Valley Name', 'slope']].rename(columns={'slope': 'slope hillslope erosion area'}),
    on='Valley Name'
)

In [12]:
terrain_attrs_erosionarea

,Valley Name,drainage area (km),curvature,slope all erosion area,slope fluvial erosion area,slope hillslope erosion area
0,Boulder,4.8292,-0.177390,24.383635,13.129799,24.623217
1,Coleman,14.4686,-0.434224,29.269404,14.665625,30.930820
2,Deming,11.4728,-0.909835,32.205973,17.398120,35.902484
3,Easton,4.6428,-0.523807,23.214705,13.850847,25.290174
4,Mazama,13.2585,-0.542106,38.080653,14.885384,40.345574
5,Park,9.9767,-0.581930,27.603668,13.988308,28.891268
6,Rainbow,12.7558,-0.219914,26.842443,9.159866,30.543619
7,Squak,2.8501,-0.070804,27.016209,20.797605,27.268083
8,Talum,3.4425,0.016154,25.749386,21.434826,26.014518
9,Thunder,6.9373,-0.186646,22.356305,20.572939,22.647718


### Load longitudinal slope measurements

In [13]:
pd.read_csv("outputs/slopes.csv")['Valley Name'].unique()

array(['Boulder', 'Coleman', 'Deming', 'Easton', 'Mazama', 'Park',
       'Rainbow', 'Squak', 'Talum', 'Thunder'], dtype=object)

In [14]:
long_slopes_df = pd.read_csv("outputs/slopes.csv")

slopes_df_pivoted = long_slopes_df.drop(columns=['Unnamed: 0']).pivot('Valley Name', columns=['year', 'measurement'], values='slope')

long_slopes_df_list = [
    pd.DataFrame(slopes_df_pivoted[1947]['Between observed glacial max and LIA']).reset_index().rename(columns={'Between observed glacial max and LIA': 'longitudinal slope limited 1947'}),
    pd.DataFrame(slopes_df_pivoted[2015]['Between observed glacial max and LIA']).reset_index().rename(columns={'Between observed glacial max and LIA': 'longitudinal slope limited 2015'}),
    pd.DataFrame(slopes_df_pivoted[1947]['All measurable area']).reset_index().rename(columns={'All measurable area': 'longitudinal slope 1947'}),
    pd.DataFrame(slopes_df_pivoted[2015]['All measurable area']).reset_index().rename(columns={'All measurable area': 'longitudinal slope 2015'})
]

cleaned_longitudinal_slope_df = reduce(lambda x, y: pd.merge(x, y, on = 'Valley Name'), long_slopes_df_list)

cleaned_longitudinal_slope_df

/tmp/ipykernel_106507/2864520677.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.pivot will be keyword-only.
  slopes_df_pivoted = long_slopes_df.drop(columns=['Unnamed: 0']).pivot('Valley Name', columns=['year', 'measurement'], values='slope')


,Valley Name,longitudinal slope limited 1947,longitudinal slope limited 2015,longitudinal slope 1947,longitudinal slope 2015
0,Boulder,10.760857,10.789069,10.760857,10.789069
1,Coleman,7.138006,7.216633,7.138006,7.216633
2,Deming,6.586342,6.531251,6.670474,6.792494
3,Easton,9.685916,9.818451,9.685916,9.818451
4,Mazama,8.421598,8.447960,7.573736,7.657434
5,Park,5.538246,5.469994,5.538246,5.469994
6,Rainbow,6.810080,6.435605,5.208286,5.029647
7,Squak,12.434124,12.352979,12.434124,12.352979
8,Talum,16.205344,16.461573,16.205344,16.461573
9,Thunder,11.489185,11.599056,11.489185,11.599056


### Load lithology data

In [15]:
lithology_df = pd.read_csv("outputs/lithology.csv")

In [16]:
lithology_df['nonigneous fraction'] = lithology_df['AREA']
lithology_df = lithology_df[['Valley Name', 'nonigneous fraction']]

In [17]:
lithology_df

,Valley Name,nonigneous fraction
0,Boulder,0.323432
1,Coleman,0.778584
2,Deming,1.000000
3,Easton,0.999963
4,Mazama,0.885783
5,Park,0.698900
6,Rainbow,0.979177
7,Squak,0.264717
8,Talum,0.133058
9,Thunder,0.268690


## Load gross volume change measurements

### Load net volume change measurements

In [18]:
net_measurements = pd.read_pickle("outputs/xdem_whole_mountain_combined/dv_df_by_valley.pickle")
neg_measurements = pd.read_pickle("outputs/xdem_whole_mountain_combined/thresh_neg_dv_df_by_valley.pickle")
pos_measurements = pd.read_pickle("outputs/xdem_whole_mountain_combined/thresh_pos_dv_df_by_valley.pickle")

In [19]:
for df in [net_measurements, neg_measurements, pos_measurements]:
    df['Valley Name'] = df['name']
    df['time interval'] = df['index']

    df = df[[
        'time interval',
        'Valley Name',
        'dh',
        'area',
        'volume',
        'bounding',
        'n_pixels',
        'start_time',
        'end_time',
        'time_difference_years',
        'Annual Mass Wasted',
        'volumetric_uncertainty',
        'Upper CI',
        'Lower CI',
        'Average Date',   
    ]].reset_index()

#### Replace numbers for the Intensive Observation Areas with the case study calculations (more accurate)

In [20]:
pd.concat([pd.read_pickle(f) for f in glob.glob("outputs/larger_area/threshold_pos_dv_df/*.pickle")]).dropna(subset='Annual Mass Wasted')

,index,dh,area,volume,n_pixels,volumetric_uncertainty,start_time,end_time,time_difference_years,Annual Mass Wasted,Upper CI,Lower CI,Average Date,valley
0,"(1947-09-14, 1979-10-06]",1.284267,420556.0,540106.371414,105139.0,290428.876490,1947-09-14,1979-10-06,32,16878.324107,25954.226497,7802.421716,1963-09-25,Deming
1,"(1979-10-06, 2015-09-01]",1.227055,421564.0,517282.194449,105391.0,75238.730561,1979-10-06,2015-09-01,36,14368.949846,16458.914584,12278.985108,1997-09-18,Deming
0,"(1947-09-14, 1979-10-06]",0.909135,274884.0,249906.742868,68721.0,35808.898631,1947-09-14,1979-10-06,32,7809.585715,8928.613797,6690.557632,1963-09-25,Coleman
1,"(1979-10-06, 2015-09-01]",0.970181,277020.0,268759.504359,69255.0,71039.465266,1979-10-06,2015-09-01,36,7465.541788,9438.860267,5492.223308,1997-09-18,Coleman
0,"(1947-09-14, 1979-10-06]",0.688509,879160.0,605309.404180,219790.0,949277.087184,1947-09-14,1979-10-06,32,18915.918881,48580.827855,-10748.990094,1963-09-25,Mazama
1,"(1979-10-06, 2015-09-01]",0.485227,936920.0,454619.065796,234230.0,192580.313311,1979-10-06,2015-09-01,36,12628.307383,17977.760531,7278.854236,1997-09-18,Mazama
0,"(1947-09-14, 1977-09-27]",1.425404,669836.0,954786.802142,167459.0,504448.711830,1947-09-14,1977-09-27,30,31826.226738,48641.183799,15011.269677,1962-09-21,Rainbow
1,"(1977-09-27, 2015-09-01]",0.601524,669836.0,402922.187430,167459.0,6133.574464,1977-09-27,2015-09-01,38,10603.215459,10764.625313,10441.805604,1996-09-14,Rainbow


In [21]:
ls -lah outputs/larger_area/bounding_dv_df/

total 20K
drwxrwxr-x.  2 elilouis elilouis   92 Aug 23  2022 ./
drwxrwxr-x. 16 elilouis elilouis 4.0K Sep  6  2022 ../
-rw-rw-r--.  1 elilouis elilouis 2.3K Apr 17 14:18 Coleman.pickle
-rw-rw-r--.  1 elilouis elilouis 2.3K Apr 17 14:17 Deming.pickle
-rw-rw-r--.  1 elilouis elilouis 2.3K Apr 17 14:20 Mazama.pickle
-rw-rw-r--.  1 elilouis elilouis 2.3K Apr 17 14:21 Rainbow.pickle


In [22]:
casestudy_netmeasurements_bounding = pd.concat([pd.read_pickle(f) for f in glob.glob("outputs/larger_area/bounding_dv_df/*.pickle")]).dropna(subset='Annual Mass Wasted')
# casestudy_negmeasurements_bounding = pd.concat([pd.read_pickle(f) for f in glob.glob("outputs/larger_area/threshold_neg_dv_df/*.pickle")]).dropna(subset='Annual Mass Wasted')
# casestudy_posmeasurements_bounding = pd.concat([pd.read_pickle(f) for f in glob.glob("outputs/larger_area/threshold_pos_dv_df/*.pickle")]).dropna(subset='Annual Mass Wasted')

casestudy_netmeasurements_bounding['bounding'] = True
casestudy_netmeasurements = pd.concat([pd.read_pickle(f) for f in glob.glob("outputs/larger_area/dv_df/*.pickle")]).dropna(subset='Annual Mass Wasted')
casestudy_netmeasurements['bounding'] = False
casestudy_netmeasurements = pd.concat([casestudy_netmeasurements, casestudy_netmeasurements_bounding])

# for df in []
casestudy_netmeasurements['Valley Name'] = casestudy_netmeasurements['valley']
casestudy_netmeasurements['time interval'] = casestudy_netmeasurements['index']
casestudy_netmeasurements = casestudy_netmeasurements[[
    'time interval',
    'Valley Name',
    'dh',
    'area',
    'volume',
    'bounding',
    'n_pixels',
    'start_time',
    'end_time',
    'time_difference_years',
    'Annual Mass Wasted',
    'volumetric_uncertainty',
    'Upper CI',
    'Lower CI',
    'Average Date',   
]].reset_index()

In [23]:
net_measurements = pd.concat([
    net_measurements[~net_measurements['Valley Name'].isin(['Deming', 'Mazama', 'Coleman',' Rainbow'])],
    casestudy_netmeasurements[casestudy_netmeasurements['Valley Name'].isin(['Deming', 'Mazama', 'Coleman',' Rainbow'])]
])

#### Name time intervals

In [24]:
date_interval_to_named_interval = {
    pd.Interval(pd.Timestamp(1947,9,14), pd.Timestamp(1977,9,27)): 'advance',
    pd.Interval(pd.Timestamp(1947,9,14), pd.Timestamp(1979,10,6)): 'advance',

    pd.Interval(pd.Timestamp(1977,9,27), pd.Timestamp(2015,9,1)): 'retreat',
    pd.Interval(pd.Timestamp(1979,10,6), pd.Timestamp(2015,9,1)): 'retreat',

    pd.Interval(pd.Timestamp(1947,9,14), pd.Timestamp(2015,9,1)): 'bounding'
}


net_measurements['named interval'] = net_measurements['time interval'].apply(date_interval_to_named_interval.get)

In [25]:
len(net_measurements['Valley Name'].unique())

10

## Merge Datasets

### Merge volume measurements and Streamstats watershed area data

In [26]:
net_measurements = net_measurements.merge(
    streamstats_gdf[['Valley Name', 'watershed area (square m)',	'watershed area (square km)']].reset_index(drop=True),
    on = 'Valley Name'
)

In [27]:
len(net_measurements['Valley Name'].unique())

10

### Merge in Terrain Attributes data (attributes of erosion polygon area)

In [28]:
net_measurements = net_measurements.merge(
    terrain_attrs_erosionarea,
    on='Valley Name'
)

In [29]:
len(net_measurements['Valley Name'].unique())

10

### Merge in glacier change data

In [30]:
net_measurements = net_measurements.merge(
    glacier_change_df, on='Valley Name'
)

In [31]:
len(net_measurements['Valley Name'].unique())

10

### Merge in lithology data

In [32]:
net_measurements.head(3)

,index,dh,area,volume,bounding,name,n_pixels,volumetric_uncertainty,start_time,end_time,time_difference_years,Annual Mass Wasted,Upper CI,Lower CI,Average Date,Annual Incision Rate,Incision Lower CI,Incision Upper CI,Valley Name,time interval,named interval,watershed area (square m),watershed area (square km),drainage area (km),curvature,slope all erosion area,slope fluvial erosion area,slope hillslope erosion area,glacial advance absolute,glacial retreat absolute,glacial advance and retreat absolute,glacial area 1947,glacial area 1977/79,glacial area 2015
0,"(1947-09-14, 1977-09-27]",-2.868870,346872.0,-9.951305e+05,False,Park,86718.0,402839.197932,1947-09-14,1977-09-27,30,-33171.017205,-19743.043940,-46598.990469,1962-09-21,-0.095629,-0.134341,-0.056917,Park,"(1947-09-14, 1977-09-27]",advance,9.948008e+06,9.948008,9.9767,-0.58193,27.603668,13.988308,28.891268,0.554,-0.39,0.944,2.743,3.298,2.908
1,"(1977-09-27, 2015-09-01]",-0.809723,346872.0,-2.808702e+05,False,Park,86718.0,15983.419752,1977-09-27,2015-09-01,38,-7391.321647,-6970.705338,-7811.937956,1996-09-14,-0.021308,-0.022521,-0.020096,Park,"(1977-09-27, 2015-09-01]",retreat,9.948008e+06,9.948008,9.9767,-0.58193,27.603668,13.988308,28.891268,0.554,-0.39,0.944,2.743,3.298,2.908
2,"(1947-09-14, 2015-09-01]",-3.490514,580512.0,-2.026285e+06,True,Park,145128.0,297791.398374,1947-09-14,2015-09-01,68,-29798.308964,-25419.023694,-34177.594235,1981-09-07,-0.051331,-0.058875,-0.043787,Park,"(1947-09-14, 2015-09-01]",bounding,9.948008e+06,9.948008,9.9767,-0.58193,27.603668,13.988308,28.891268,0.554,-0.39,0.944,2.743,3.298,2.908


In [33]:
net_measurements = net_measurements.merge(
    lithology_df, on='Valley Name'
)

##### Assign Nooksack river fork

In [34]:
net_measurements['Fork of the Nooksack River'] = 'Does not drain to Nooksack River'
net_measurements.loc[net_measurements["Valley Name"] == "Coleman", 'Fork of the Nooksack River'] = "North Fork"
net_measurements.loc[net_measurements["Valley Name"] == "Deming", 'Fork of the Nooksack River'] = "Middle Fork"
net_measurements.loc[net_measurements["Valley Name"] == "Mazama", 'Fork of the Nooksack River'] = "North Fork"
net_measurements.loc[net_measurements["Valley Name"] == "Thunder", 'Fork of the Nooksack River'] = "Middle Fork"

In [35]:
len(net_measurements['Valley Name'].unique())

10

## Calculate sediment yields from volume measurements

In [36]:
net_measurements['sediment yield (t / yr)'] = - net_measurements['Annual Mass Wasted'] * (1 - porosity) * density_kg_per_cubic_meter / kg_per_metric_ton
net_measurements['Upper CI sediment yield'] = - net_measurements['Upper CI'] * (1 - porosity) * density_kg_per_cubic_meter / kg_per_metric_ton
net_measurements['Lower CI sediment yield'] = - net_measurements['Lower CI'] * (1 - porosity) * density_kg_per_cubic_meter / kg_per_metric_ton

net_measurements['sediment yield normalized (t / km^2 / yr)'] = net_measurements['sediment yield (t / yr)'] / net_measurements['watershed area (square km)']
net_measurements['Upper CI sediment yield normalized'] = net_measurements['Upper CI sediment yield'] / net_measurements['watershed area (square km)']
net_measurements['Lower CI sediment yield normalized'] = net_measurements['Lower CI sediment yield'] / net_measurements['watershed area (square km)']

net_measurements['Annual Mass Wasted normalized'] = net_measurements['Annual Mass Wasted'] / (net_measurements['watershed area (square km)']*(1000**2))
net_measurements['Upper CI normalized'] = net_measurements['Upper CI'] / (net_measurements['watershed area (square km)']*(1000**2))
net_measurements['Lower CI normalized'] = net_measurements['Lower CI'] / (net_measurements['watershed area (square km)']*(1000**2))

# Plot Sediment Yield (1947-2015)

In [37]:
yield_domain = [-50, 168.99999999999997]
volume_domain = [-29585.79881656805/1000, 100]

In [38]:
valley_sorting = ['Coleman','Deming','Rainbow','Mazama','Park','Easton','Boulder','Thunder','Squak','Talum']

In [47]:
fig_width=200
fig_height=300
yield_domain = [-11.8310517529, 8.4507512520868]
volume_domain = [-0.007, 0.005]

src_bounding = net_measurements[net_measurements['named interval'] == 'bounding'].drop(columns=['time interval']).drop(columns='index')

src_bounding['Annual Mass Wasted normalized'] = -src_bounding['Annual Mass Wasted normalized']
src_bounding['Lower CI normalized'] = -src_bounding['Lower CI normalized']
src_bounding['Upper CI normalized'] = -src_bounding['Upper CI normalized']
src_bounding['sediment yield normalized (kt / km^2 / yr)'] = src_bounding['sediment yield normalized (t / km^2 / yr)']/1000

base = alt.Chart(src_bounding).encode(alt.X("Valley Name", axis=alt.Axis(labelAngle = -60), sort=valley_sorting)).properties(width=fig_width, height=fig_height)
volume = base.mark_bar().encode(
    alt.Y("Annual Mass Wasted normalized:Q", title='Specifc sediment yield (m/yr)', sort=valley_sorting, scale=alt.Scale(
        domain=volume_domain, 
        nice=False
    )),
    alt.Color("Fork of the Nooksack River:N")
)
sedyield = base.mark_bar().encode(
    alt.Y("sediment yield normalized (kt / km^2 / yr):Q", sort=valley_sorting, scale=alt.Scale(
            domain=yield_domain, 
            nice=False
        ), 
        title='Specific sediment yield (kt / km² / yr)'
    ),
    alt.Color("Fork of the Nooksack River:N", scale = alt.Scale(
        domain = [
            'Does not drain to Nooksack River',
            "North Fork",
            "Middle Fork"
        ],
        range = ['#282828', '#808080' , '#DCDCDC'])
    )
)
error_bars = base.mark_bar(width=2, color='black', stroke='white').encode(
        alt.Y("Lower CI normalized:Q", scale=alt.Scale(
                domain=volume_domain, 
                nice=False
            ), 
            title='', axis=alt.Axis(labels=False)
        ),
    alt.Y2("Upper CI normalized:Q", title='')
)
alt.layer(
    volume, 
    sedyield, 
    error_bars
).resolve_scale(y='independent').configure_legend(
    titleFontSize=12,
    labelFontSize=12,
    orient='top'
).configure_axis(
    labelFontSize=12,
    titleFontSize=14,
    titleFontWeight='normal'
)

alt.LayerChart(...)

In [46]:


fig_width=200
fig_height=300
yield_domain = [-50, 168.99999999999997]
volume_domain = [-29585.79881656805/1000, 100]

src = net_measurements.drop(columns=['time interval']).drop(columns='index')
src['Annual Mass Wasted'] = -src['Annual Mass Wasted']/1000
src['Lower CI'] = -src['Lower CI']/1000
src['Upper CI'] = -src['Upper CI']/1000
src['Lower CI sediment yield'] = -src['Lower CI sediment yield']/1000
src['Upper CI sediment yield'] = -src['Upper CI sediment yield']/1000

src['sediment yield (t / yr)'] = src['sediment yield (t / yr)']/1000

src_bounding = src[src['named interval'] == 'bounding']
base = alt.Chart(src_bounding).encode(alt.X("Valley Name", axis=alt.Axis(labelAngle = -60), sort=valley_sorting)).properties(width=fig_width, height=fig_height)
volume = base.mark_bar().encode(
    alt.Y("Annual Mass Wasted:Q", title='Annual sediment yield (10³ m³/yr)', sort=valley_sorting, scale=alt.Scale(domain=volume_domain, nice=False)),
    alt.Color("Fork of the Nooksack River:N")
)
sedyield = base.mark_bar().encode(
    alt.Y("sediment yield (t / yr):Q", sort=valley_sorting, scale=alt.Scale(domain=yield_domain, nice=False), title='Annual sediment yield (kt / yr)'),
    alt.Color("Fork of the Nooksack River:N", scale = alt.Scale(
        domain = [
            'Does not drain to Nooksack River',
            "North Fork",
            "Middle Fork"
        ],
        range = ['#282828', '#808080' , '#DCDCDC'])
    )
)
error_bars = base.mark_bar(width=2, color='black', stroke='white').encode(
        alt.Y("Lower CI:Q", scale=alt.Scale(domain=volume_domain, nice=False), title='', axis=alt.Axis(labels=False)),
    alt.Y2("Upper CI:Q", title='')
)
bounding_chart = alt.layer(volume, sedyield, error_bars).resolve_scale(y='independent')

src_advance = src[src['named interval'] == 'advance']
base = alt.Chart(src_advance).encode(alt.X("Valley Name", axis=alt.Axis(labelAngle = -60), sort=valley_sorting)).properties(width=fig_width, height=fig_height)
volume = base.mark_bar().encode(
    alt.Y("Annual Mass Wasted:Q", title='Annual sediment yield (10³ m³/yr)', sort=valley_sorting, scale=alt.Scale(domain=volume_domain, nice=False)),
    alt.Color("Fork of the Nooksack River:N")
)
sedyield = base.mark_bar().encode(
    alt.Y("sediment yield (t / yr):Q", sort=valley_sorting, scale=alt.Scale(domain=yield_domain, nice=False), title='Annual sediment yield (kt / yr)'),
    alt.Color("Fork of the Nooksack River:N")
)
error_bars = base.mark_bar(width=2, color='black').encode(
        alt.Y("Lower CI:Q", scale=alt.Scale(domain=volume_domain, nice=False), title='', axis=alt.Axis(labels=False)),
    alt.Y2("Upper CI:Q", title='')
)
advance_chart = alt.layer(volume, sedyield, error_bars).resolve_scale(y='independent')


src_retreat = src[src['named interval'] == 'retreat']
base = alt.Chart(src_retreat).encode(alt.X("Valley Name", axis=alt.Axis(labelAngle = -60), sort=valley_sorting)).properties(width=fig_width, height=fig_height)
volume = base.mark_bar().encode(
    alt.Y("Annual Mass Wasted:Q", title='Annual sediment yield (10³ m³/yr)', sort=valley_sorting, scale=alt.Scale(domain=volume_domain, nice=False)),
    alt.Color("Fork of the Nooksack River:N")
)
sedyield = base.mark_bar().encode(
    alt.Y("sediment yield (t / yr):Q", sort=valley_sorting, scale=alt.Scale(domain=yield_domain, nice=False), title='Annual sediment yield (kt / yr)'),
    alt.Color("Fork of the Nooksack River:N")
)
error_bars = base.mark_bar(width=2, color='black').encode(
    alt.Y("Lower CI:Q", scale=alt.Scale(domain=volume_domain, nice=False), title='', axis=alt.Axis(labels=False)),
    alt.Y2("Upper CI:Q", title='')
)
retreat_chart = alt.layer(volume, sedyield, error_bars).resolve_scale(y='independent')

(
    bounding_chart | 
    advance_chart | 
    retreat_chart
).configure_legend(titleFontSize=12, labelFontSize=12, orient='top').configure_axis(labelFontSize=12, titleFontSize=14, titleFontWeight='normal')

alt.HConcatChart(...)

## Save data to csv

In [41]:
src[[
'Valley Name',
'Fork of the Nooksack River',
'named interval',
'Annual Mass Wasted',
'Lower CI',
'Upper CI',
'Lower CI sediment yield',
'Upper CI sediment yield',
'sediment yield (t / yr)',
]].to_csv('outputs/final_figures_data/volumes_and_yields_per_valley.csv')

### Merge in longitudinal slope data

In [42]:
net_measurements = net_measurements.merge(
    cleaned_longitudinal_slope_df, 
    on='Valley Name'
)

## Keep only the bounding data

In [43]:
net_measurements = net_measurements[net_measurements['named interval'] == 'bounding']

In [44]:
net_measurements = net_measurements[[
    'Valley Name',
    'watershed area (square km)', # A, could also use the column "drainage area (km)" instead
    'longitudinal slope 2015', # S_c
    'slope hillslope erosion area', # S_h
    'glacial retreat absolute', # ∆A_g
    'glacial area 1977/79', #A_g
    'nonigneous fraction',
    'sediment yield (t / yr)', #Q_s
    'sediment yield normalized (t / km^2 / yr)',
    'Upper CI sediment yield',
    'Lower CI sediment yield',
    'Upper CI sediment yield normalized',
    'Lower CI sediment yield normalized'
]]
net_measurements['glacial retreat absolute'] = -net_measurements['glacial retreat absolute']
net_measurements['glacial retreat relative'] = net_measurements['glacial retreat absolute'] / net_measurements['glacial area 1977/79']

In [45]:
net_measurements

,Valley Name,watershed area (square km),longitudinal slope 2015,slope hillslope erosion area,glacial retreat absolute,glacial area 1977/79,nonigneous fraction,sediment yield (t / yr),sediment yield normalized (t / km^2 / yr),Upper CI sediment yield,Lower CI sediment yield,Upper CI sediment yield normalized,Lower CI sediment yield normalized,glacial retreat relative
2,Park,9.948008,5.469994,28.891268,0.390,3.298,0.698900,50359.142150,5062.233550,42958.150043,57760.134256,4318.266339,5806.200761,0.118253
5,Rainbow,12.856056,5.029647,30.543619,0.687,2.699,0.979177,71479.504877,5559.986920,60931.176310,82027.833444,4739.492025,6380.481816,0.254539
8,Boulder,5.634194,10.789069,24.623217,0.802,6.441,0.323432,10853.029108,1926.278878,-6741.808620,28447.866836,-1196.587922,5049.145677,0.124515
11,Squak,3.947675,12.352979,27.268083,0.359,2.324,0.264717,-12214.960038,-3094.216009,-21340.515618,-3089.404458,-5405.843724,-782.588293,0.154475
14,Thunder,7.110903,11.599056,22.647718,0.180,0.970,0.268690,4638.899525,652.364292,-8914.914513,18192.713563,-1253.696456,2558.425039,0.185567
17,Talum,3.130098,16.461573,26.014518,0.482,2.390,0.133058,-20898.738407,-6676.704548,-31873.658018,-9923.818796,-10182.959052,-3170.450043,0.201674
20,Easton,4.698693,9.818451,25.290174,0.185,2.897,0.999963,22240.343929,4733.304511,16171.407889,28309.279968,3441.682294,6024.926728,0.063859
23,Deming,11.589236,6.792494,35.902484,0.369,5.126,1.000000,88536.471267,7639.543190,79986.661393,97086.281141,6901.806065,8377.280315,0.071986
26,Coleman,13.854967,7.216633,30.930820,0.658,10.916,0.778584,75451.031568,5445.775025,68313.039877,82589.023260,4930.581315,5960.968735,0.060278
29,Mazama,13.196766,7.657434,40.345574,0.563,5.583,0.885783,79213.853641,6002.520137,59225.996129,99201.711154,4487.917429,7517.122844,0.100842


# Plot Scatterplots

## Prep Data

### rename columns (for plotting convenience)

In [46]:
net_measurements = net_measurements.rename(columns={
    'sediment yield (t / yr)': 'Sediment Yield (ton/yr)',
    'watershed area (square km)': 'Drainage area (square km)',
    'longitudinal slope 2015': 'Channel slope',
    'slope hillslope erosion area': 'Hillslope domain slope',
    'glacial retreat absolute': 'Glacial retreat area (km²)',
    'nonigneous fraction': 'Nonigneous fraction',  
    'sediment yield normalized (t / km^2 / yr)' : 'Sediment Yield (ton/km²/yr)' 
})


net_measurements

,Valley Name,Drainage area (square km),Channel slope,Hillslope domain slope,Glacial retreat area (km²),glacial area 1977/79,Nonigneous fraction,Sediment Yield (ton/yr),Sediment Yield (ton/km²/yr),Upper CI sediment yield,Lower CI sediment yield,Upper CI sediment yield normalized,Lower CI sediment yield normalized,glacial retreat relative
2,Park,9.948008,5.469994,28.891268,0.390,3.298,0.698900,50359.142150,5062.233550,42958.150043,57760.134256,4318.266339,5806.200761,0.118253
5,Rainbow,12.856056,5.029647,30.543619,0.687,2.699,0.979177,71479.504877,5559.986920,60931.176310,82027.833444,4739.492025,6380.481816,0.254539
8,Boulder,5.634194,10.789069,24.623217,0.802,6.441,0.323432,10853.029108,1926.278878,-6741.808620,28447.866836,-1196.587922,5049.145677,0.124515
11,Squak,3.947675,12.352979,27.268083,0.359,2.324,0.264717,-12214.960038,-3094.216009,-21340.515618,-3089.404458,-5405.843724,-782.588293,0.154475
14,Thunder,7.110903,11.599056,22.647718,0.180,0.970,0.268690,4638.899525,652.364292,-8914.914513,18192.713563,-1253.696456,2558.425039,0.185567
17,Talum,3.130098,16.461573,26.014518,0.482,2.390,0.133058,-20898.738407,-6676.704548,-31873.658018,-9923.818796,-10182.959052,-3170.450043,0.201674
20,Easton,4.698693,9.818451,25.290174,0.185,2.897,0.999963,22240.343929,4733.304511,16171.407889,28309.279968,3441.682294,6024.926728,0.063859
23,Deming,11.589236,6.792494,35.902484,0.369,5.126,1.000000,88536.471267,7639.543190,79986.661393,97086.281141,6901.806065,8377.280315,0.071986
26,Coleman,13.854967,7.216633,30.930820,0.658,10.916,0.778584,75451.031568,5445.775025,68313.039877,82589.023260,4930.581315,5960.968735,0.060278
29,Mazama,13.196766,7.657434,40.345574,0.563,5.583,0.885783,79213.853641,6002.520137,59225.996129,99201.711154,4487.917429,7517.122844,0.100842


### Convert slopes to tan of slopes

In [47]:
print([f for f in net_measurements.columns if 'slope' in f])
for slope_col in [f for f in net_measurements.columns if 'slope' in f]:
    net_measurements[slope_col + ' (degrees)'] = net_measurements[slope_col]
    net_measurements[slope_col] = np.tan(np.deg2rad(net_measurements[slope_col]))

net_measurements

['Channel slope', 'Hillslope domain slope']


,Valley Name,Drainage area (square km),Channel slope,Hillslope domain slope,Glacial retreat area (km²),glacial area 1977/79,Nonigneous fraction,Sediment Yield (ton/yr),Sediment Yield (ton/km²/yr),Upper CI sediment yield,Lower CI sediment yield,Upper CI sediment yield normalized,Lower CI sediment yield normalized,glacial retreat relative,Channel slope (degrees),Hillslope domain slope (degrees)
2,Park,9.948008,0.095761,0.551831,0.390,3.298,0.698900,50359.142150,5062.233550,42958.150043,57760.134256,4318.266339,5806.200761,0.118253,5.469994,28.891268
5,Rainbow,12.856056,0.088010,0.590071,0.687,2.699,0.979177,71479.504877,5559.986920,60931.176310,82027.833444,4739.492025,6380.481816,0.254539,5.029647,30.543619
8,Boulder,5.634194,0.190562,0.458326,0.802,6.441,0.323432,10853.029108,1926.278878,-6741.808620,28447.866836,-1196.587922,5049.145677,0.124515,10.789069,24.623217
11,Squak,3.947675,0.219004,0.515433,0.359,2.324,0.264717,-12214.960038,-3094.216009,-21340.515618,-3089.404458,-5405.843724,-782.588293,0.154475,12.352979,27.268083
14,Thunder,7.110903,0.205253,0.417237,0.180,0.970,0.268690,4638.899525,652.364292,-8914.914513,18192.713563,-1253.696456,2558.425039,0.185567,11.599056,22.647718
17,Talum,3.130098,0.295484,0.488046,0.482,2.390,0.133058,-20898.738407,-6676.704548,-31873.658018,-9923.818796,-10182.959052,-3170.450043,0.201674,16.461573,26.014518
20,Easton,4.698693,0.173062,0.472488,0.185,2.897,0.999963,22240.343929,4733.304511,16171.407889,28309.279968,3441.682294,6024.926728,0.063859,9.818451,25.290174
23,Deming,11.589236,0.119110,0.723945,0.369,5.126,1.000000,88536.471267,7639.543190,79986.661393,97086.281141,6901.806065,8377.280315,0.071986,6.792494,35.902484
26,Coleman,13.854967,0.126624,0.599218,0.658,10.916,0.778584,75451.031568,5445.775025,68313.039877,82589.023260,4930.581315,5960.968735,0.060278,7.216633,30.930820
29,Mazama,13.196766,0.134449,0.849430,0.563,5.583,0.885783,79213.853641,6002.520137,59225.996129,99201.711154,4487.917429,7517.122844,0.100842,7.657434,40.345574


## Plot sediment yield vs explanatory variables

In [48]:
circles_y = alt.Chart().mark_point(size=100, strokeWidth=2).encode(
    alt.Y('Sediment Yield (ton/yr):Q'),
    alt.Color('Valley Name:N')
)
points_y = alt.Chart().mark_circle(size=110).encode(
    alt.Y('Sediment Yield (ton/yr):Q'),
    alt.Color('Valley Name:N')
)
bars_y = alt.Chart().mark_line().encode(
    alt.Y('Lower CI sediment yield'),
    alt.Y2('Upper CI sediment yield'),
    alt.Color('Valley Name:N')
)

darea = alt.layer(
    points_y.transform_filter(alt.FieldGTEPredicate('Sediment Yield (ton/yr)', 0)).encode(alt.X('Drainage area (square km):Q')), 
    circles_y.transform_filter(alt.FieldLTPredicate('Sediment Yield (ton/yr)', 0)).encode(alt.X('Drainage area (square km):Q')), 
    bars_y.encode(alt.X('Drainage area (square km):Q')), 
    data=net_measurements
).properties(
    width=200, height=200
)

channelslope = alt.layer(
    points_y.transform_filter(alt.FieldGTEPredicate('Sediment Yield (ton/yr)', 0)).encode(alt.X('Channel slope:Q')), 
    circles_y.transform_filter(alt.FieldLTPredicate('Sediment Yield (ton/yr)', 0)).encode(alt.X('Channel slope:Q')), 
    bars_y.encode(alt.X('Channel slope:Q', scale=alt.Scale(zero=False))),
    data=net_measurements
).properties(
    width=200, height=200
)

hillslope = alt.layer(
    points_y.transform_filter(alt.FieldGTEPredicate('Sediment Yield (ton/yr)', 0)).encode(alt.X('Hillslope domain slope:Q')), 
    circles_y.transform_filter(alt.FieldLTPredicate('Sediment Yield (ton/yr)', 0)).encode(alt.X('Hillslope domain slope:Q')), 
    bars_y.encode(alt.X('Hillslope domain slope:Q', scale=alt.Scale(zero=False))),
    data=net_measurements
).properties(
    width=200, height=200
)

glacialretreat = alt.layer(
    points_y.transform_filter(alt.FieldGTEPredicate('Sediment Yield (ton/yr)', 0)).encode(alt.X('Glacial retreat area (km²):Q')), 
    circles_y.transform_filter(alt.FieldLTPredicate('Sediment Yield (ton/yr)', 0)).encode(alt.X('Glacial retreat area (km²):Q')), 
    bars_y.encode(alt.X('Glacial retreat area (km²):Q')),
    data=net_measurements
).properties(
    width=200, height=200
)

lithology = alt.layer(
    points_y.transform_filter(alt.FieldGTEPredicate('Sediment Yield (ton/yr)', 0)).encode(alt.X('Nonigneous fraction:Q')), 
    circles_y.transform_filter(alt.FieldLTPredicate('Sediment Yield (ton/yr)', 0)).encode(alt.X('Nonigneous fraction:Q')), 
    bars_y.encode(alt.X('Nonigneous fraction:Q')),
    data=net_measurements
).properties(
    width=200, height=200
)

darea | channelslope | hillslope | glacialretreat | lithology

alt.HConcatChart(...)

## Plot specific sediment yield vs explanatory variables

In [49]:
circles_y = alt.Chart().mark_point(size=100, strokeWidth=2).encode(
    alt.Y('Sediment Yield (ton/km²/yr):Q'),
    alt.Color('Valley Name:N')
)
points_y = alt.Chart().mark_circle(size=110).encode(
    alt.Y('Sediment Yield (ton/km²/yr):Q'),
    alt.Color('Valley Name:N')
)
bars_y = alt.Chart().mark_line().encode(
    alt.Y('Lower CI sediment yield normalized'),
    alt.Y2('Upper CI sediment yield normalized'),
    alt.Color('Valley Name:N')
)

darea_normalized = alt.layer(
    points_y.transform_filter(alt.FieldGTEPredicate('Sediment Yield (ton/km²/yr)', 0)).encode(alt.X('Drainage area (square km):Q')), 
    circles_y.transform_filter(alt.FieldLTPredicate('Sediment Yield (ton/km²/yr)', 0)).encode(alt.X('Drainage area (square km):Q')), 
    bars_y.encode(alt.X('Drainage area (square km):Q')), 
    data=net_measurements
).properties(
    width=200, height=200
)

channelslope_normalized = alt.layer(
    points_y.transform_filter(alt.FieldGTEPredicate('Sediment Yield (ton/km²/yr)', 0)).encode(alt.X('Channel slope:Q')), 
    circles_y.transform_filter(alt.FieldLTPredicate('Sediment Yield (ton/km²/yr)', 0)).encode(alt.X('Channel slope:Q')), 
    bars_y.encode(alt.X('Channel slope:Q', scale=alt.Scale(zero=False))),
    data=net_measurements
).properties(
    width=200, height=200
)

hillslope_normalized = alt.layer(
    points_y.transform_filter(alt.FieldGTEPredicate('Sediment Yield (ton/km²/yr)', 0)).encode(alt.X('Hillslope domain slope:Q')), 
    circles_y.transform_filter(alt.FieldLTPredicate('Sediment Yield (ton/km²/yr)', 0)).encode(alt.X('Hillslope domain slope:Q')), 
    bars_y.encode(alt.X('Hillslope domain slope:Q', scale=alt.Scale(zero=False))),
    data=net_measurements
).properties(
    width=200, height=200
)

glacialretreat_normalized = alt.layer(
    points_y.transform_filter(alt.FieldGTEPredicate('Sediment Yield (ton/km²/yr)', 0)).encode(alt.X('Glacial retreat area (km²):Q')), 
    circles_y.transform_filter(alt.FieldLTPredicate('Sediment Yield (ton/km²/yr)', 0)).encode(alt.X('Glacial retreat area (km²):Q')), 
    bars_y.encode(alt.X('Glacial retreat area (km²):Q')),
    data=net_measurements
).properties(
    width=200, height=200
)

lithology_normalized = alt.layer(
    points_y.transform_filter(alt.FieldGTEPredicate('Sediment Yield (ton/km²/yr)', 0)).encode(alt.X('Nonigneous fraction:Q')), 
    circles_y.transform_filter(alt.FieldLTPredicate('Sediment Yield (ton/km²/yr)', 0)).encode(alt.X('Nonigneous fraction:Q')), 
    bars_y.encode(alt.X('Nonigneous fraction:Q')),
    data=net_measurements
).properties(
    width=200, height=200
)

darea_normalized | channelslope_normalized | hillslope_normalized | glacialretreat_normalized | lithology_normalized

alt.HConcatChart(...)

## Plot explanatory variables interactions

In [56]:
(
    alt.Chart(net_measurements).mark_circle(size=100).encode(
        alt.Y('Drainage area (square km):Q', title='Drainage area (km²)', scale=alt.Scale(zero=False)),
        alt.X("Nonigneous fraction:Q",),
        alt.Color("Valley Name:N")
    ).properties(
        width=200, height=200
    ) | \
    alt.Chart(net_measurements).mark_circle(size=100).encode(
        alt.Y('Channel slope:Q', scale=alt.Scale(zero=False)),
        alt.X("Nonigneous fraction:Q",),
        alt.Color("Valley Name:N")
    ).properties(
        width=200, height=200
    ) | \
    alt.Chart(net_measurements).mark_circle(size=100).encode(
        alt.Y('Hillslope domain slope:Q', scale=alt.Scale(zero=False)),
        alt.X("Nonigneous fraction:Q",),
        alt.Color("Valley Name:N")
    ).properties(
        width=200, height=200
    ) | \
    alt.Chart(net_measurements).mark_circle(size=100).encode(
        alt.Y('Glacial retreat area (km²):Q', scale=alt.Scale(zero=False)),
        alt.X("Nonigneous fraction:Q",),
        alt.Color("Valley Name:N")
    ).properties(
        width=200, height=200
    )


).configure_legend(
    titleFontSize=22, labelFontSize=22, orient='top'
).configure_axis(
    labelFontSize=16, titleFontSize=16
).configure_title(
    fontSize=22
)

alt.HConcatChart(...)

# Modeling

In [284]:
model_data = net_measurements.copy()

## Save Model Data to CSV

In [285]:
model_data.to_csv("outputs/modeling_powerlaw_data.csv")

## Remove net depositional data points

In [286]:
model_data = model_data[model_data["Sediment Yield (ton/yr)"] > 0]

In [287]:
model_data['Valley Name'].unique()

array(['Park', 'Rainbow', 'Boulder', 'Thunder', 'Easton', 'Deming',
       'Coleman', 'Mazama'], dtype=object)

## Remove two outlier basins

In [288]:
# model_data = model_data[~model_data['Valley Name'].isin(['Boulder', 'Thunder'])]

In [289]:
model_data['Valley Name'].unique()

array(['Park', 'Rainbow', 'Boulder', 'Thunder', 'Easton', 'Deming',
       'Coleman', 'Mazama'], dtype=object)

## Create Models

In [290]:
parameters_dict_powerlaw = {}
parameters_dict_linear = {}

parameters_dict_powerlaw_normalized = {}
parameters_dict_linear_normalized = {}

### Define Power law models

In [291]:
def model_1_powerlaw(x, k, l):
    """
    Args:
        x (float): drainage area
    """
    return k*x**l

def model_2_powerlaw(x, k, m):
    """
    Args:
        x (float): channel slope
    """
    return k*x**m

def model_3_powerlaw(x, k, n):
    """
    Args:
        x (float): hillslope slope
    """
    return k*x**n

def model_4_powerlaw(x, k, p):
    """
    Args:
        x (float): glacier area change
    """
    return k*x**p

def model_5_powerlaw(x, k, q):
    """
    Args:
        x (float): nonigneous fraction
    """
    return k*x**q

def model_6_powerlaw(x, k, l, n):
    """
    Args:
        x (list(float))): [drainage area, hillslope slope]
    """
    return k*(x[0]**l)*(x[1]**n)

def model_7_powerlaw(x, k, l, n, p):
    """
    Args:
        x (list(float))): [drainage area, hillslope slope, glacier area change]
    """
    return k*(x[0]**l)*(x[1]**n)*(x[2]**p)

# def model7(x, k, l, m, n):
#     """
#     Args:
#         x (list(float))): [drainage area, channel slope, hillslope slope]
#     """
#     return k*(x[0]**l)*(x[1]**m)*(x[2]**n)

# def model8(x, k, l, m, n, p):
#     """
#     Args:
#         x (list(float))): [drainage area, channel slope, hillslope slope, glacier area change]
#     """
#     return k*(x[0]**l)*(x[1]**m)*(x[2]**n)*(x[3]**p)

### Define linear models

In [292]:
def model_1_linear(x, l, i):
    """
    Args:
        x (float): drainage area
    """
    return l*x + i

def model_2_linear(x, m, i):
    """
    Args:
        x (float): channel slope
    """
    return m*x + i

def model_3_linear(x, n, i):
    """
    Args:
        x (float): hillslope slope
    """
    return n*x

def model_4_linear(x, p, i):
    """
    Args:
        x (float): glacier area change
    """
    return p*x + i

def model_5_linear(x, q, i):
    """
    Args:
        x (float): nonigneous fraction
    """
    return q*x + i

def model_6_linear(x, l, n, i):
    """
    Args:
        x (list(float))): [drainage area, hillslope slope]
    """
    return l*x[0] + n*x[1] + i

def model_7_linear(x, l, n, p, i):
    """
    Args:
        x (list(float))): [drainage area, hillslope slope, glacier area change]
    """
    return l*x[0] + n*x[1] + p*x[2] + i

## Run Models

#### Model 1

In [293]:
popt_powerlaw

[106.36, -2.9]

In [294]:
model_1_data = model_data.copy()



popt_powerlaw, pcov_powerlaw = scipy.optimize.curve_fit(
    model_1_powerlaw, 
    model_1_data['Drainage area (square km)'].to_list(),
    model_1_data['Sediment Yield (ton/yr)'].to_list()
)

popt_powerlaw = [385.93, 2.0287]

popt_linear, pcov_linear = scipy.optimize.curve_fit(
    model_1_linear, 
    model_1_data['Drainage area (square km)'].to_list(),
    model_1_data['Sediment Yield (ton/yr)'].to_list()
)

model_1_data['Power law predicted sediment yield (ton/yr)'] = model_1_data.apply(
    lambda row: model_1_powerlaw(row['Drainage area (square km)'], popt_powerlaw[0], popt_powerlaw[1]),
    axis=1
)

model_1_data['Linear predicted sediment yield (ton/yr)'] = model_1_data.apply(
    lambda row: model_1_linear(row['Drainage area (square km)'], popt_linear[0], popt_linear[1]),
    axis=1
)

parameters_dict_powerlaw[1] = popt_powerlaw
parameters_dict_linear[1] = popt_linear

model_1_plot_linear = r2_plot(model_1_data, 'Linear predicted sediment yield (ton/yr)', 'Sediment Yield (ton/yr)', limit=100000)
model_1_plot_powerlaw = r2_plot(model_1_data, 'Power law predicted sediment yield (ton/yr)', 'Sediment Yield (ton/yr)', limit=100000)

model_1_plot_linear | model_1_plot_powerlaw

alt.HConcatChart(...)

#### Model 1 SSY

In [295]:
model_1_data_normalized = model_data.copy()

popt_powerlaw, pcov_powerlaw = scipy.optimize.curve_fit(
    model_1_powerlaw, 
    model_1_data_normalized['Drainage area (square km)'].to_list(),
    model_1_data_normalized['Sediment Yield (ton/km²/yr)'].to_list()
)

popt_linear, pcov_linear = scipy.optimize.curve_fit(
    model_1_linear, 
    model_1_data_normalized['Drainage area (square km)'].to_list(),
    model_1_data_normalized['Sediment Yield (ton/km²/yr)'].to_list()
)

model_1_data_normalized['Power law predicted sediment yield (ton/km²/yr)'] = model_1_data_normalized.apply(
    lambda row: model_1_powerlaw(row['Drainage area (square km)'], popt_powerlaw[0], popt_powerlaw[1]),
    axis=1
)

model_1_data_normalized['Linear predicted sediment yield (ton/km²/yr)'] = model_1_data_normalized.apply(
    lambda row: model_1_linear(row['Drainage area (square km)'], popt_linear[0], popt_linear[1]),
    axis=1
)

parameters_dict_powerlaw_normalized[1] = popt_powerlaw
parameters_dict_linear_normalized[1] = popt_linear

model_1_plot_linear_normalized = r2_plot(model_1_data_normalized, 'Linear predicted sediment yield (ton/km²/yr)', 'Sediment Yield (ton/km²/yr)', limit=8000)
model_1_plot_powerlaw_normalized = r2_plot(model_1_data_normalized, 'Power law predicted sediment yield (ton/km²/yr)', 'Sediment Yield (ton/km²/yr)', limit=8000)

model_1_plot_linear_normalized | model_1_plot_powerlaw_normalized

alt.HConcatChart(...)

#### Model 2

In [296]:
list(model_2_data['Channel slope'])

0.09576051794295204
0.08801007946008708
0.19056247852466418
0.20525334499275494
0.17306165731237597
0.11910990865642232
0.12662431872566676
0.13444887662357244

0.13444887662357244

In [297]:
list(model_2_data['Sediment Yield (ton/yr)'])

50359.14214972888
71479.50487685582
10853.02910758811
4638.899524866833
22240.34392858765
88536.47126671483
75451.03156821251
79213.85364142025

79213.85364142025

In [298]:
model_2_data = model_data.copy()

popt_powerlaw, pcov_powerlaw = scipy.optimize.curve_fit(
    model_2_powerlaw, 
    model_2_data['Channel slope'].to_list(),
    model_2_data['Sediment Yield (ton/yr)'].to_list(),
    [106, -2.9],
    method = 'trf'
)

popt_powerlaw = [106.36, -2.9]

popt_linear, pcov_linear = scipy.optimize.curve_fit(
    model_2_linear, 
    model_2_data['Channel slope'].to_list(),
    model_2_data['Sediment Yield (ton/yr)'].to_list()
)

model_2_data['Power law predicted sediment yield (ton/yr)'] = model_2_data.apply(
    lambda row: model_2_powerlaw(row['Channel slope'], popt_powerlaw[0], popt_powerlaw[1]),
    axis=1
)

model_2_data['Linear predicted sediment yield (ton/yr)'] = model_2_data.apply(
    lambda row: model_2_linear(row['Channel slope'], popt_linear[0], popt_linear[1]),
    axis=1
)

parameters_dict_powerlaw[2] = popt_powerlaw
parameters_dict_linear[2] = popt_linear

model_2_plot_linear = r2_plot(model_2_data, 'Linear predicted sediment yield (ton/yr)', 'Sediment Yield (ton/yr)', limit=100000)
model_2_plot_powerlaw = r2_plot(model_2_data, 'Power law predicted sediment yield (ton/yr)', 'Sediment Yield (ton/yr)', limit=100000)

model_2_plot_linear | model_2_plot_powerlaw

alt.HConcatChart(...)

#### Model 2 SSY

In [299]:
model_2_data_normalized = model_data.copy()

popt_powerlaw, pcov_powerlaw = scipy.optimize.curve_fit(
    model_2_powerlaw, 
    model_2_data_normalized['Channel slope'].to_list(),
    model_2_data_normalized['Sediment Yield (ton/km²/yr)'].to_list()
)

popt_linear, pcov_linear = scipy.optimize.curve_fit(
    model_2_linear, 
    model_2_data_normalized['Channel slope'].to_list(),
    model_2_data_normalized['Sediment Yield (ton/km²/yr)'].to_list()
)

model_2_data_normalized['Power law predicted sediment yield (ton/km²/yr)'] = model_2_data_normalized.apply(
    lambda row: model_2_powerlaw(row['Channel slope'], popt_powerlaw[0], popt_powerlaw[1]),
    axis=1
)

model_2_data_normalized['Linear predicted sediment yield (ton/km²/yr)'] = model_2_data_normalized.apply(
    lambda row: model_2_linear(row['Channel slope'], popt_linear[0], popt_linear[1]),
    axis=1
)

parameters_dict_powerlaw_normalized[2] = popt_powerlaw
parameters_dict_linear_normalized[2] = popt_linear

model_2_plot_linear_normalized = r2_plot(model_2_data_normalized, 'Linear predicted sediment yield (ton/km²/yr)', 'Sediment Yield (ton/km²/yr)', limit=8000)
model_2_plot_powerlaw_normalized = r2_plot(model_2_data_normalized, 'Power law predicted sediment yield (ton/km²/yr)', 'Sediment Yield (ton/km²/yr)', limit=8000)

model_2_plot_linear_normalized | model_2_plot_powerlaw_normalized

alt.HConcatChart(...)

#### Model 3

In [300]:
model_3_data = model_data.copy()



popt_powerlaw, pcov_powerlaw = scipy.optimize.curve_fit(
    model_3_powerlaw, 
    model_3_data['Hillslope domain slope'].to_list(),
    model_3_data['Sediment Yield (ton/yr)'].to_list()
)

popt_powerlaw = [318055, 3.9142]

popt_linear, pcov_linear = scipy.optimize.curve_fit(
    model_3_linear, 
    model_3_data['Hillslope domain slope'].to_list(),
    model_3_data['Sediment Yield (ton/yr)'].to_list()
)

model_3_data['Power law predicted sediment yield (ton/yr)'] = model_3_data.apply(
    lambda row: model_3_powerlaw(row['Hillslope domain slope'], popt_powerlaw[0], popt_powerlaw[1]),
    axis=1
)

model_3_data['Linear predicted sediment yield (ton/yr)'] = model_3_data.apply(
    lambda row: model_3_linear(row['Hillslope domain slope'], popt_linear[0], popt_linear[1]),
    axis=1
)

parameters_dict_powerlaw[3] = popt_powerlaw
parameters_dict_linear[3] = popt_linear

model_3_plot_linear = r2_plot(model_3_data, 'Linear predicted sediment yield (ton/yr)', 'Sediment Yield (ton/yr)', limit=100000)
model_3_plot_powerlaw = r2_plot(model_3_data, 'Power law predicted sediment yield (ton/yr)', 'Sediment Yield (ton/yr)', limit=100000)

model_3_plot_linear | model_3_plot_powerlaw

/home/elilouis/mambaforge/envs/hsfm/lib/python3.9/site-packages/scipy/optimize/_minpack_py.py:906: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',


alt.HConcatChart(...)

#### Model 3 SSY

In [301]:
model_3_data_normalized = model_data.copy()

popt_powerlaw, pcov_powerlaw = scipy.optimize.curve_fit(
    model_3_powerlaw, 
    model_3_data_normalized['Hillslope domain slope'].to_list(),
    model_3_data_normalized['Sediment Yield (ton/km²/yr)'].to_list()
)

popt_linear, pcov_linear = scipy.optimize.curve_fit(
    model_3_linear, 
    model_3_data_normalized['Hillslope domain slope'].to_list(),
    model_3_data_normalized['Sediment Yield (ton/km²/yr)'].to_list()
)

model_3_data_normalized['Power law predicted sediment yield (ton/km²/yr)'] = model_3_data_normalized.apply(
    lambda row: model_3_powerlaw(row['Hillslope domain slope'], popt_powerlaw[0], popt_powerlaw[1]),
    axis=1
)

model_3_data_normalized['Linear predicted sediment yield (ton/km²/yr)'] = model_3_data_normalized.apply(
    lambda row: model_3_linear(row['Hillslope domain slope'], popt_linear[0], popt_linear[1]),
    axis=1
)

parameters_dict_powerlaw_normalized[3] = popt_powerlaw
parameters_dict_linear_normalized[3] = popt_linear

model_3_plot_linear_normalized = r2_plot(model_3_data_normalized, 'Linear predicted sediment yield (ton/km²/yr)', 'Sediment Yield (ton/km²/yr)', limit=8000)
model_3_plot_powerlaw_normalized = r2_plot(model_3_data_normalized, 'Power law predicted sediment yield (ton/km²/yr)', 'Sediment Yield (ton/km²/yr)', limit=8000)

model_3_plot_linear_normalized | model_3_plot_powerlaw_normalized

/home/elilouis/mambaforge/envs/hsfm/lib/python3.9/site-packages/scipy/optimize/_minpack_py.py:906: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',


alt.HConcatChart(...)

#### Model 4

In [302]:
model_4_data = model_data.copy()



popt_powerlaw, pcov_powerlaw = scipy.optimize.curve_fit(
    model_4_powerlaw, 
    model_4_data['Glacial retreat area (km²)'].to_list(),
    model_4_data['Sediment Yield (ton/yr)'].to_list()
)

popt_linear, pcov_linear = scipy.optimize.curve_fit(
    model_4_linear, 
    model_4_data['Glacial retreat area (km²)'].to_list(),
    model_4_data['Sediment Yield (ton/yr)'].to_list()
)

model_4_data['Power law predicted sediment yield (ton/yr)'] = model_4_data.apply(
    lambda row: model_4_powerlaw(row['Glacial retreat area (km²)'], popt_powerlaw[0], popt_powerlaw[1]),
    axis=1
)

model_4_data['Linear predicted sediment yield (ton/yr)'] = model_4_data.apply(
    lambda row: model_4_linear(row['Glacial retreat area (km²)'], popt_linear[0], popt_linear[1]),
    axis=1
)

parameters_dict_powerlaw[4] = popt_powerlaw
parameters_dict_linear[4] = popt_linear

model_4_plot_linear = r2_plot(model_4_data, 'Linear predicted sediment yield (ton/yr)', 'Sediment Yield (ton/yr)', limit=100000)
model_4_plot_powerlaw = r2_plot(model_4_data, 'Power law predicted sediment yield (ton/yr)', 'Sediment Yield (ton/yr)', limit=100000)

model_4_plot_linear | model_4_plot_powerlaw

alt.HConcatChart(...)

#### Model 4 SSY

In [303]:
model_4_data_normalized = model_data.copy()



popt_powerlaw, pcov_powerlaw = scipy.optimize.curve_fit(
    model_4_powerlaw, 
    model_4_data_normalized['Glacial retreat area (km²)'].to_list(),
    model_4_data_normalized['Sediment Yield (ton/km²/yr)'].to_list()
)

popt_linear, pcov_linear = scipy.optimize.curve_fit(
    model_4_linear, 
    model_4_data_normalized['Glacial retreat area (km²)'].to_list(),
    model_4_data_normalized['Sediment Yield (ton/km²/yr)'].to_list()
)

model_4_data_normalized['Power law predicted sediment yield (ton/km²/yr)'] = model_4_data_normalized.apply(
    lambda row: model_4_powerlaw(row['Glacial retreat area (km²)'], popt_powerlaw[0], popt_powerlaw[1]),
    axis=1
)

model_4_data_normalized['Linear predicted sediment yield (ton/km²/yr)'] = model_4_data_normalized.apply(
    lambda row: model_4_linear(row['Glacial retreat area (km²)'], popt_linear[0], popt_linear[1]),
    axis=1
)

parameters_dict_powerlaw_normalized[4] = popt_powerlaw
parameters_dict_linear_normalized[4] = popt_linear

model_4_plot_linear_normalized = r2_plot(model_4_data_normalized, 'Linear predicted sediment yield (ton/km²/yr)', 'Sediment Yield (ton/km²/yr)', limit=8000)
model_4_plot_powerlaw_normalized = r2_plot(model_4_data_normalized, 'Power law predicted sediment yield (ton/km²/yr)', 'Sediment Yield (ton/km²/yr)', limit=8000)

model_4_plot_linear_normalized | model_4_plot_powerlaw_normalized

alt.HConcatChart(...)

#### Model 5

In [304]:
model_5_data = model_data.copy()



popt_powerlaw, pcov_powerlaw = scipy.optimize.curve_fit(
    model_5_powerlaw, 
    model_5_data['Nonigneous fraction'].to_list(),
    model_5_data['Sediment Yield (ton/yr)'].to_list()
)

popt_linear, pcov_linear = scipy.optimize.curve_fit(
    model_5_linear, 
    model_5_data['Nonigneous fraction'].to_list(),
    model_5_data['Sediment Yield (ton/yr)'].to_list()
)

model_5_data['Power law predicted sediment yield (ton/yr)'] = model_5_data.apply(
    lambda row: model_5_powerlaw(row['Nonigneous fraction'], popt_powerlaw[0], popt_powerlaw[1]),
    axis=1
)

model_5_data['Linear predicted sediment yield (ton/yr)'] = model_5_data.apply(
    lambda row: model_5_linear(row['Nonigneous fraction'], popt_linear[0], popt_linear[1]),
    axis=1
)

parameters_dict_powerlaw[5] = popt_powerlaw
parameters_dict_linear[5] = popt_linear

model_5_plot_linear = r2_plot(model_5_data, 'Linear predicted sediment yield (ton/yr)', 'Sediment Yield (ton/yr)', limit=100000)
model_5_plot_powerlaw = r2_plot(model_5_data, 'Power law predicted sediment yield (ton/yr)', 'Sediment Yield (ton/yr)', limit=100000)

model_5_plot_linear | model_5_plot_powerlaw

alt.HConcatChart(...)

#### Model 5 SSY

In [305]:
model_5_data_normalized = model_data.copy()



popt_powerlaw, pcov_powerlaw = scipy.optimize.curve_fit(
    model_5_powerlaw, 
    model_5_data_normalized['Nonigneous fraction'].to_list(),
    model_5_data_normalized['Sediment Yield (ton/km²/yr)'].to_list()
)

popt_linear, pcov_linear = scipy.optimize.curve_fit(
    model_5_linear, 
    model_5_data_normalized['Nonigneous fraction'].to_list(),
    model_5_data_normalized['Sediment Yield (ton/km²/yr)'].to_list()
)

model_5_data_normalized['Power law predicted sediment yield (ton/km²/yr)'] = model_5_data_normalized.apply(
    lambda row: model_5_powerlaw(row['Nonigneous fraction'], popt_powerlaw[0], popt_powerlaw[1]),
    axis=1
)

model_5_data_normalized['Linear predicted sediment yield (ton/km²/yr)'] = model_5_data_normalized.apply(
    lambda row: model_5_linear(row['Nonigneous fraction'], popt_linear[0], popt_linear[1]),
    axis=1
)

parameters_dict_powerlaw_normalized[5] = popt_powerlaw
parameters_dict_linear_normalized[5] = popt_linear

model_5_plot_linear_normalized = r2_plot(model_5_data_normalized, 'Linear predicted sediment yield (ton/km²/yr)', 'Sediment Yield (ton/km²/yr)', limit=8000)
model_5_plot_powerlaw_normalized = r2_plot(model_5_data_normalized, 'Power law predicted sediment yield (ton/km²/yr)', 'Sediment Yield (ton/km²/yr)', limit=8000)

model_5_plot_linear_normalized | model_5_plot_powerlaw_normalized

alt.HConcatChart(...)

#### Model 6

In [306]:
model_6_data = model_data.copy()



popt_powerlaw, pcov_powerlaw = scipy.optimize.curve_fit(
    model_6_powerlaw, 
    np.array([
        model_6_data['Drainage area (square km)'].to_list(),
        model_6_data['Hillslope domain slope'].to_list(),
    ]),
    model_6_data['Sediment Yield (ton/yr)'].to_list()
)

popt_linear, pcov_linear = scipy.optimize.curve_fit(
    model_6_linear, 
    np.array([
        model_6_data['Drainage area (square km)'].to_list(),
        model_6_data['Hillslope domain slope'].to_list(),
    ]),
    model_6_data['Sediment Yield (ton/yr)'].to_list()
)

model_6_data['Power law predicted sediment yield (ton/yr)'] = model_6_data.apply(
    lambda row: model_6_powerlaw(
        (
            row['Drainage area (square km)'],
            row['Hillslope domain slope']
        ), popt_powerlaw[0], popt_powerlaw[1], popt_powerlaw[2]
    ),
    axis=1
)

model_6_data['Linear predicted sediment yield (ton/yr)'] = model_6_data.apply(
    lambda row: model_6_linear(
        (
            row['Drainage area (square km)'],
            row['Hillslope domain slope']
        ), popt_linear[0], popt_linear[1], popt_linear[2]
    ),
    axis=1
)

parameters_dict_powerlaw[6] = popt_powerlaw
parameters_dict_linear[6] = popt_linear

model_6_plot_linear = r2_plot(model_6_data, 'Linear predicted sediment yield (ton/yr)', 'Sediment Yield (ton/yr)', limit=100000)
model_6_plot_powerlaw = r2_plot(model_6_data, 'Power law predicted sediment yield (ton/yr)', 'Sediment Yield (ton/yr)', limit=100000)

model_6_plot_powerlaw_error_bars = alt.Chart(model_6_data).mark_rule(color='#1f77b4').encode(
    alt.X('Power law predicted sediment yield (ton/yr)'),
    alt.Y('Lower CI sediment yield'),
    alt.Y2('Upper CI sediment yield')
)

model_6_plot_linear | model_6_plot_powerlaw+model_6_plot_powerlaw_error_bars

alt.HConcatChart(...)

#### Model 6 SSY

In [307]:
model_6_data_normalized = model_data.copy()



popt_powerlaw, pcov_powerlaw = scipy.optimize.curve_fit(
    model_6_powerlaw, 
    np.array([
        model_6_data_normalized['Drainage area (square km)'].to_list(),
        model_6_data_normalized['Hillslope domain slope'].to_list(),
    ]),
    model_6_data_normalized['Sediment Yield (ton/km²/yr)'].to_list()
)

popt_linear, pcov_linear = scipy.optimize.curve_fit(
    model_6_linear, 
    np.array([
        model_6_data_normalized['Drainage area (square km)'].to_list(),
        model_6_data_normalized['Hillslope domain slope'].to_list(),
    ]),
    model_6_data_normalized['Sediment Yield (ton/km²/yr)'].to_list()
)

model_6_data_normalized['Power law predicted sediment yield (ton/km²/yr)'] = model_6_data_normalized.apply(
    lambda row: model_6_powerlaw(
        (
            row['Drainage area (square km)'],
            row['Hillslope domain slope']
        ), popt_powerlaw[0], popt_powerlaw[1], popt_powerlaw[2]
    ),
    axis=1
)

model_6_data_normalized['Linear predicted sediment yield (ton/km²/yr)'] = model_6_data_normalized.apply(
    lambda row: model_6_linear(
        (
            row['Drainage area (square km)'],
            row['Hillslope domain slope']
        ), popt_linear[0], popt_linear[1], popt_linear[2]
    ),
    axis=1
)

parameters_dict_powerlaw_normalized[6] = popt_powerlaw
parameters_dict_linear_normalized[6] = popt_linear

model_6_plot_linear_normalized = r2_plot(model_6_data_normalized, 'Linear predicted sediment yield (ton/km²/yr)', 'Sediment Yield (ton/km²/yr)', limit=8000)
model_6_plot_powerlaw_normalized = r2_plot(model_6_data_normalized, 'Power law predicted sediment yield (ton/km²/yr)', 'Sediment Yield (ton/km²/yr)', limit=8000)



model_6_plot_powerlaw_error_bars_normalized = alt.Chart(model_6_data_normalized).mark_rule(color='#1f77b4').encode(
    alt.X('Power law predicted sediment yield (ton/km²/yr)', title=''),
    alt.Y('Lower CI sediment yield normalized'),
    alt.Y2('Upper CI sediment yield normalized')
)
model_6_plot_linear_normalized | model_6_plot_powerlaw_normalized+model_6_plot_powerlaw_error_bars_normalized

alt.HConcatChart(...)

#### Model 7


In [308]:
model_7_data = model_data.copy()



popt_powerlaw, pcov_powerlaw = scipy.optimize.curve_fit(
    model_7_powerlaw, 
    np.array([
        model_7_data['Drainage area (square km)'].to_list(),
        model_7_data['Hillslope domain slope'].to_list(),
        model_7_data['Glacial retreat area (km²)'].to_list(),
    ]),
    model_7_data['Sediment Yield (ton/yr)'].to_list()
)

popt_linear, pcov_linear = scipy.optimize.curve_fit(
    model_7_linear, 
    np.array([
        model_7_data['Drainage area (square km)'].to_list(),
        model_7_data['Hillslope domain slope'].to_list(),
        model_7_data['Glacial retreat area (km²)'].to_list(),
    ]),
    model_7_data['Sediment Yield (ton/yr)'].to_list()
)

model_7_data['Power law predicted sediment yield (ton/yr)'] = model_7_data.apply(
    lambda row: model_7_powerlaw(
        (
            row['Drainage area (square km)'],
            row['Hillslope domain slope'],
            row['Glacial retreat area (km²)'],
        ), popt_powerlaw[0], popt_powerlaw[1], popt_powerlaw[2], popt_powerlaw[3]
    ),
    axis=1
)

model_7_data['Linear predicted sediment yield (ton/yr)'] = model_7_data.apply(
    lambda row: model_7_linear(
        (
            row['Drainage area (square km)'],
            row['Hillslope domain slope'],
            row['Glacial retreat area (km²)'],
        ), popt_linear[0], popt_linear[1], popt_linear[2], popt_linear[3]
    ),
    axis=1
)

parameters_dict_powerlaw[7] = popt_powerlaw
parameters_dict_linear[7] = popt_linear

model_7_plot_linear = r2_plot(model_7_data, 'Linear predicted sediment yield (ton/yr)', 'Sediment Yield (ton/yr)', limit=100000)
model_7_plot_powerlaw = r2_plot(model_7_data, 'Power law predicted sediment yield (ton/yr)', 'Sediment Yield (ton/yr)', limit=100000)

model_7_plot_linear | model_7_plot_powerlaw

/tmp/ipykernel_124082/3509343863.py:48: RuntimeWarning: overflow encountered in multiply
  return k*(x[0]**l)*(x[1]**n)*(x[2]**p)


alt.HConcatChart(...)

#### Model 7 SSY

In [309]:
model_7_data_normalized = model_data.copy()



popt_powerlaw, pcov_powerlaw = scipy.optimize.curve_fit(
    model_7_powerlaw, 
    np.array([
        model_7_data_normalized['Drainage area (square km)'].to_list(),
        model_7_data_normalized['Hillslope domain slope'].to_list(),
        model_7_data_normalized['Glacial retreat area (km²)'].to_list(),
    ]),
    model_7_data_normalized['Sediment Yield (ton/km²/yr)'].to_list()
)

popt_linear, pcov_linear = scipy.optimize.curve_fit(
    model_7_linear, 
    np.array([
        model_7_data_normalized['Drainage area (square km)'].to_list(),
        model_7_data_normalized['Hillslope domain slope'].to_list(),
        model_7_data_normalized['Glacial retreat area (km²)'].to_list(),
    ]),
    model_7_data_normalized['Sediment Yield (ton/km²/yr)'].to_list()
)

model_7_data_normalized['Power law predicted sediment yield (ton/km²/yr)'] = model_7_data_normalized.apply(
    lambda row: model_7_powerlaw(
        (
            row['Drainage area (square km)'],
            row['Hillslope domain slope'],
            row['Glacial retreat area (km²)'],
        ), popt_powerlaw[0], popt_powerlaw[1], popt_powerlaw[2], popt_powerlaw[3]
    ),
    axis=1
)

model_7_data_normalized['Linear predicted sediment yield (ton/km²/yr)'] = model_7_data_normalized.apply(
    lambda row: model_7_linear(
        (
            row['Drainage area (square km)'],
            row['Hillslope domain slope'],
            row['Glacial retreat area (km²)'],
        ), popt_linear[0], popt_linear[1], popt_linear[2], popt_linear[3]
    ),
    axis=1
)

parameters_dict_powerlaw_normalized[7] = popt_powerlaw
parameters_dict_linear_normalized[7] = popt_linear

model_7_plot_linear_normalized = r2_plot(model_7_data_normalized, 'Linear predicted sediment yield (ton/km²/yr)', 'Sediment Yield (ton/km²/yr)', limit=8000)
model_7_plot_powerlaw_normalized = r2_plot(model_7_data_normalized, 'Power law predicted sediment yield (ton/km²/yr)', 'Sediment Yield (ton/km²/yr)', limit=8000)

model_7_plot_linear_normalized | model_7_plot_powerlaw_normalized

/tmp/ipykernel_124082/3509343863.py:48: RuntimeWarning: overflow encountered in multiply
  return k*(x[0]**l)*(x[1]**n)*(x[2]**p)


alt.HConcatChart(...)

# Plot Model Results

In [310]:
def add_props(plot, params, data, model_n, predicted_col, nse_label = "NSE: "):
    return plot.properties(
        title={
            'subtitle': [
                str([np.round(var, 2) for var in params]),
                nse_label + str(np.round(nse(data['Sediment Yield (ton/yr)'], data[predicted_col]), 3))],
            'text': f"Model {str(model_n)}"
        }
    )

## Multivariate models - Observed vs Predicted Plots

In [311]:
model_6_plot_powerlaw_normalized

alt.LayerChart(...)

In [312]:
model_6_ssy_results = add_props(
    model_6_plot_powerlaw_normalized + model_6_plot_powerlaw_error_bars_normalized, parameters_dict_powerlaw_normalized[6], model_6_data_normalized, 6, predicted_col='Power law predicted sediment yield (ton/km²/yr)', nse_label='NSE (Power law Model): '
).encode(
    alt.X(title="Predicted sediment yield (ton/km²/yr)"),
    alt.Y(title='Observed sediment yield (ton/km²/yr)')
)

model_6_results = add_props(
    model_6_plot_powerlaw + model_6_plot_powerlaw_error_bars, parameters_dict_powerlaw[6], model_6_data, 6, predicted_col='Power law predicted sediment yield (ton/yr)', nse_label='NSE (Power law Model): '
).encode(
    alt.X(title="Predicted sediment yield (ton/yr)"),
    alt.Y(title='Observed sediment yield (ton/yr)')
)

(model_6_ssy_results | model_6_results).configure_legend(
    titleFontSize=22, labelFontSize=22, orient='right'
).configure_axis(
    labelFontSize=16, titleFontSize=16
).configure_title(
    fontSize=22
)

alt.HConcatChart(...)

## Single variate models - Observed vs Variabel Plots

### Sediment Yield

In [313]:
def add_props_2models(plot, data, model_n, params_linear, params_powerlaw):
    return plot.properties(
        width=200, height=200,
        title={
            'subtitle': [
                "Parameters (Linear model): " + str([np.round(var, 2) for var in params_linear]),
                "Parameters (Power law model): " + str([np.round(var, 2) for var in params_powerlaw]),
                "NSE (Linear model): " + str(np.round(nse(data['Sediment Yield (ton/yr)'], data['Linear predicted sediment yield (ton/yr)']), 2)),
                # "r² (Linear model): " + str(np.round(r2_score(data['Sediment Yield (ton/yr)'], data['Linear predicted sediment yield (ton/yr)']), 2)),
                "NSE (Power law model): " + str(np.round(nse(data['Sediment Yield (ton/yr)'], data['Power law predicted sediment yield (ton/yr)']), 2))
            ],
            'text': f"Model {str(model_n)}"
        }
    )

#### Model 1

In [314]:

domain_space = pd.Series(np.linspace(0,14,100))
model_1_df = pd.DataFrame({
    'domain': domain_space,
    'Linear model': domain_space.apply(lambda x: model_1_linear(x, parameters_dict_linear[1][0], parameters_dict_linear[1][1])),
    'Power law model': domain_space.apply(lambda x: model_1_powerlaw(x, parameters_dict_powerlaw[1][0], parameters_dict_powerlaw[1][1])) 
})

linear_model = alt.Chart(model_1_df).mark_line(color='grey', strokeWidth=2, opacity=0.5).encode(
    alt.X('domain', title='Drainage area (km²)'),
    alt.Y('Linear model', scale=alt.Scale(domain=(-40000, 120000)))
)

powerlaw_model = alt.Chart(model_1_df).mark_line(color='black', strokeWidth=2, opacity=1).encode(
    alt.X('domain'),
    alt.Y('Power law model')
)

model_1_scatterplot = (linear_model + powerlaw_model + darea)

model_1_scatterplot = add_props_2models(model_1_scatterplot, model_1_data, 1, parameters_dict_linear[1], parameters_dict_powerlaw[1])
model_1_scatterplot

alt.LayerChart(...)

#### Model 2

In [315]:

domain_space = pd.Series(np.linspace(0.05,0.30,100))
model_2_df = pd.DataFrame({
    'domain': domain_space,
    'Linear model': domain_space.apply(lambda x: model_2_linear(x, parameters_dict_linear[2][0], parameters_dict_linear[2][1])),
    'Power law model': domain_space.apply(lambda x: model_2_powerlaw(x, parameters_dict_powerlaw[2][0], parameters_dict_powerlaw[2][1])) 
})

linear_model = alt.Chart(model_2_df).mark_line(color='grey', strokeWidth=2, opacity=0.5).encode(
    alt.X('domain', title='Channel slope'),
    alt.Y('Linear model', scale=alt.Scale(domain=(-40000, 120000), clamp=True))
)

powerlaw_model = alt.Chart(model_2_df).mark_line(color='black', strokeWidth=2, opacity=1).encode(
    alt.X('domain'),
    alt.Y('Power law model')
)

model_2_scatterplot = (linear_model + powerlaw_model + channelslope)

model_2_scatterplot = add_props_2models(model_2_scatterplot, model_2_data, 2, parameters_dict_linear[2], parameters_dict_powerlaw[2])

#### Model 3

In [316]:

domain_space = pd.Series(np.linspace(0.4,0.9,100))
model_3_df = pd.DataFrame({
    'domain': domain_space,
    'Linear model': domain_space.apply(lambda x: model_3_linear(x, parameters_dict_linear[3][0], parameters_dict_linear[3][1])),
    'Power law model': domain_space.apply(lambda x: model_3_powerlaw(x, parameters_dict_powerlaw[3][0], parameters_dict_powerlaw[3][1])) 
})

linear_model = alt.Chart(model_3_df).mark_line(color='grey', strokeWidth=2, opacity=0.5).encode(
    alt.X('domain', title='Hillslope domain slope'),
    alt.Y('Linear model', scale=alt.Scale(domain=(-40000, 120000)))
)

powerlaw_model = alt.Chart(model_3_df).mark_line(color='black', strokeWidth=2, opacity=1).encode(
    alt.X('domain'),
    alt.Y('Power law model')
)

model_3_scatterplot = (linear_model + powerlaw_model + hillslope)

model_3_scatterplot = add_props_2models(model_3_scatterplot, model_3_data, 3, parameters_dict_linear[3], parameters_dict_powerlaw[3])

#### Model 4

In [317]:

domain_space = pd.Series(np.linspace(0.0,0.9,100))
model_4_df = pd.DataFrame({
    'domain': domain_space,
    'Linear model': domain_space.apply(lambda x: model_4_linear(x, parameters_dict_linear[4][0], parameters_dict_linear[4][1])),
    'Power law model': domain_space.apply(lambda x: model_4_powerlaw(x, parameters_dict_powerlaw[4][0], parameters_dict_powerlaw[4][1])) 
})

linear_model = alt.Chart(model_4_df).mark_line(color='grey', strokeWidth=2, opacity=0.5).encode(
    alt.X('domain', title='Glacial retreat area (km²)'),
    alt.Y('Linear model', scale=alt.Scale(domain=(-40000, 120000)))
)

powerlaw_model = alt.Chart(model_4_df).mark_line(color='black', strokeWidth=2, opacity=1).encode(
    alt.X('domain'),
    alt.Y('Power law model')
)

# model_4_scatterplot = (linear_model + powerlaw_model + glacialretreat)
model_4_scatterplot = (linear_model + glacialretreat)

model_4_scatterplot = add_props_2models(model_4_scatterplot, model_4_data, 4, parameters_dict_linear[4], parameters_dict_powerlaw[4])

#### Model 5 

In [318]:

domain_space = pd.Series(np.linspace(0.0,1.0,100))
model_5_df = pd.DataFrame({
    'domain': domain_space,
    'Linear model': domain_space.apply(lambda x: model_5_linear(x, parameters_dict_linear[5][0], parameters_dict_linear[5][1])),
    'Power law model': domain_space.apply(lambda x: model_5_powerlaw(x, parameters_dict_powerlaw[5][0], parameters_dict_powerlaw[5][1])) 
})

linear_model = alt.Chart(model_5_df).mark_line(color='grey', strokeWidth=2, opacity=0.5).encode(
    alt.X('domain', title='Nonigneous fraction'),
    alt.Y('Linear model', scale=alt.Scale(domain=(-40000, 120000)), title='')
)

powerlaw_model = alt.Chart(model_5_df).mark_line(color='black', strokeWidth=2, opacity=1).encode(
    alt.X('domain'),
    alt.Y('Power law model')
)

model_5_scatterplot = linear_model + powerlaw_model + lithology
model_5_scatterplot = linear_model + lithology

model_5_scatterplot = add_props_2models(model_5_scatterplot, model_5_data, 5, parameters_dict_linear[5], parameters_dict_powerlaw[5])

In [319]:
(
    model_1_scatterplot | model_2_scatterplot | model_3_scatterplot | model_4_scatterplot | model_5_scatterplot
).configure_legend(
    titleFontSize=22, labelFontSize=22, orient='right'
).configure_axis(
    labelFontSize=22, titleFontSize=22
).configure_title(
    fontSize=22
)

alt.HConcatChart(...)

### Specific Sediment Yield

In [320]:
def add_props_2models_normalized(plot, data, model_n, params_linear, params_powerlaw):
    return plot.properties(
        width=200, height=200,
        title={
            'subtitle': [
                "Parameters (Linear model): " + str([np.round(var, 2) for var in params_linear]),
                "Parameters (Power law model): " + str([np.round(var, 2) for var in params_powerlaw]),
                "NSE (Linear model): " + str(np.round(nse(data['Sediment Yield (ton/km²/yr)'], data['Linear predicted sediment yield (ton/km²/yr)']), 2)),
                # "r² (Linear model): " + str(np.round(r2_score(data['Sediment Yield (ton/km²/yr)'], data['Linear predicted sediment yield (ton/km²/yr)']), 2)),
                "NSE (Power law model): " + str(np.round(nse(data['Sediment Yield (ton/km²/yr)'], data['Power law predicted sediment yield (ton/km²/yr)']), 2))
            ],
            'text': f"Model {str(model_n)}"
        }
    )

#### Model 1

In [321]:

domain_space = pd.Series(np.linspace(0,14,100))
model_1_df = pd.DataFrame({
    'domain': domain_space,
    'Linear model': domain_space.apply(lambda x: model_1_linear(x, parameters_dict_linear_normalized[1][0], parameters_dict_linear_normalized[1][1])),
    'Power law model': domain_space.apply(lambda x: model_1_powerlaw(x, parameters_dict_powerlaw_normalized[1][0], parameters_dict_powerlaw_normalized[1][1])) 
})

linear_model = alt.Chart(model_1_df).mark_line(color='grey', strokeWidth=2, opacity=0.5).encode(
    alt.X('domain', title='Drainage area (km²)'),
    alt.Y('Linear model', scale=alt.Scale(domain=(-10000, 10000)))
)

powerlaw_model = alt.Chart(model_1_df).mark_line(color='black', strokeWidth=2, opacity=1).encode(
    alt.X('domain'),
    alt.Y('Power law model')
)

model_1_scatterplot_normed = (linear_model + darea_normalized)

model_1_scatterplot_normed = add_props_2models_normalized(model_1_scatterplot_normed, model_1_data_normalized, 1, parameters_dict_linear_normalized[1], parameters_dict_powerlaw_normalized[1])

#### Model 2

In [322]:

domain_space = pd.Series(np.linspace(0.05,0.30,100))
model_2_df = pd.DataFrame({
    'domain': domain_space,
    'Linear model': domain_space.apply(lambda x: model_2_linear(x, parameters_dict_linear_normalized[2][0], parameters_dict_linear_normalized[2][1])),
    'Power law model': domain_space.apply(lambda x: model_2_powerlaw(x, parameters_dict_powerlaw_normalized[2][0], parameters_dict_powerlaw_normalized[2][1])) 
})

linear_model = alt.Chart(model_2_df).mark_line(color='grey', strokeWidth=2, opacity=0.5).encode(
    alt.X('domain', title='Channel slope'),
    alt.Y('Linear model', scale=alt.Scale(domain=(-10000, 10000), clamp=True))
)

powerlaw_model = alt.Chart(model_2_df).mark_line(color='black', strokeWidth=2, opacity=1).encode(
    alt.X('domain'),
    alt.Y('Power law model')
)

model_2_scatterplot_normed = (linear_model + channelslope_normalized)

model_2_scatterplot_normed = add_props_2models_normalized(model_2_scatterplot_normed, model_2_data_normalized, 2, parameters_dict_linear_normalized[2], parameters_dict_powerlaw_normalized[2])

#### Model 3

In [323]:

domain_space = pd.Series(np.linspace(0.4,0.9,100))
model_3_df = pd.DataFrame({
    'domain': domain_space,
    'Linear model': domain_space.apply(lambda x: model_3_linear(x, parameters_dict_linear_normalized[3][0], parameters_dict_linear_normalized[3][1])),
    'Power law model': domain_space.apply(lambda x: model_3_powerlaw(x, parameters_dict_powerlaw_normalized[3][0], parameters_dict_powerlaw_normalized[3][1])) 
})

linear_model = alt.Chart(model_3_df).mark_line(color='grey', strokeWidth=2, opacity=0.5).encode(
    alt.X('domain', title='Hillslope domain slope'),
    alt.Y('Linear model', scale=alt.Scale(domain=(-10000, 10000)))
)

powerlaw_model = alt.Chart(model_3_df).mark_line(color='black', strokeWidth=2, opacity=1).encode(
    alt.X('domain'),
    alt.Y('Power law model')
)

model_3_scatterplot_normed = (linear_model + hillslope_normalized)

model_3_scatterplot_normed = add_props_2models_normalized(model_3_scatterplot_normed, model_3_data_normalized, 3, parameters_dict_linear_normalized[3], parameters_dict_powerlaw_normalized[3])

#### Model 4

In [324]:

domain_space = pd.Series(np.linspace(0.0,0.9,100))
model_4_df = pd.DataFrame({
    'domain': domain_space,
    'Linear model': domain_space.apply(lambda x: model_4_linear(x, parameters_dict_linear_normalized[4][0], parameters_dict_linear_normalized[4][1])),
    'Power law model': domain_space.apply(lambda x: model_4_powerlaw(x, parameters_dict_powerlaw_normalized[4][0], parameters_dict_powerlaw_normalized[4][1])) 
})

linear_model = alt.Chart(model_4_df).mark_line(color='grey', strokeWidth=2, opacity=0.5).encode(
    alt.X('domain', title='Glacial retreat area (km²)'),
    alt.Y('Linear model', scale=alt.Scale(domain=(-10000, 10000)))
)

powerlaw_model = alt.Chart(model_4_df).mark_line(color='black', strokeWidth=2, opacity=1).encode(
    alt.X('domain'),
    alt.Y('Power law model')
)

model_4_scatterplot_normed = (linear_model + glacialretreat_normalized)

model_4_scatterplot_normed = add_props_2models_normalized(model_4_scatterplot_normed, model_4_data_normalized, 4, parameters_dict_linear_normalized[4], parameters_dict_powerlaw_normalized[4])

#### Model 5 

In [325]:

domain_space = pd.Series(np.linspace(0.0,1.0,100))
model_5_df = pd.DataFrame({
    'domain': domain_space,
    'Linear model': domain_space.apply(lambda x: model_5_linear(x, parameters_dict_linear_normalized[5][0], parameters_dict_linear_normalized[5][1])),
    'Power law model': domain_space.apply(lambda x: model_5_powerlaw(x, parameters_dict_powerlaw_normalized[5][0], parameters_dict_powerlaw_normalized[5][1])) 
})

linear_model = alt.Chart(model_5_df).mark_line(color='grey', strokeWidth=2, opacity=0.5).encode(
    alt.X('domain', title='Nonigneous fraction'),
    alt.Y('Linear model', scale=alt.Scale(domain=(-10000, 10000)), title='')
)

powerlaw_model = alt.Chart(model_5_df).mark_line(color='black', strokeWidth=2, opacity=1).encode(
    alt.X('domain'),
    alt.Y('Power law model')
)

model_5_scatterplot_normed = linear_model + lithology_normalized

model_5_scatterplot_normed = add_props_2models_normalized(model_5_scatterplot_normed, model_5_data_normalized, 5, parameters_dict_linear_normalized[5], parameters_dict_powerlaw_normalized[5])

In [326]:
def y_label_none(plot):
    return plot.encode(alt.Y(title="", axis=alt.Axis(labels=False)))


(
    (model_1_scatterplot.encode(alt.Y(title="Sediment Yield (ton/yr)")) | y_label_none(model_2_scatterplot) | y_label_none(model_3_scatterplot) | y_label_none(model_4_scatterplot) | y_label_none(model_5_scatterplot))
    &
    (model_1_scatterplot_normed.encode(alt.Y(title="Specific Sediment Yield (ton/km²/yr)")) | y_label_none(model_2_scatterplot_normed) | y_label_none(model_3_scatterplot_normed) | y_label_none(model_4_scatterplot_normed) | y_label_none(model_5_scatterplot_normed))
).configure_legend(
    titleFontSize=22, labelFontSize=22, orient='right'
).configure_axis(
    labelFontSize=16, titleFontSize=16
).configure_title(
    fontSize=22
)

alt.VConcatChart(...)

In [327]:
def y_label_none(plot):
    return plot.encode(alt.Y(title="", axis=alt.Axis(labels=False)))
def no_title(plot):
    return plot.properties(title="")

(
    (no_title(model_1_scatterplot).encode(alt.Y(title="Sediment Yield (ton/yr)")) | y_label_none(no_title(model_2_scatterplot)) | y_label_none(no_title(model_3_scatterplot)) | y_label_none(no_title(model_4_scatterplot)) | y_label_none(no_title(model_5_scatterplot)))
    &
    (no_title(model_1_scatterplot_normed).encode(alt.Y(title="Specific Sediment Yield (ton/km²/yr)")) | y_label_none(no_title(model_2_scatterplot_normed)) | y_label_none(no_title(model_3_scatterplot_normed)) | y_label_none(no_title(model_4_scatterplot_normed)) | y_label_none(no_title(model_5_scatterplot_normed)))
).configure_legend(
    titleFontSize=22, labelFontSize=22, orient='top'
).configure_axis(
    labelFontSize=18, titleFontSize=18
).configure_title(
    fontSize=22
)

alt.VConcatChart(...)

In [328]:
def y_label_none(plot):
    return plot.encode(alt.Y(title="", axis=alt.Axis(labels=False)))


(
    (model_1_scatterplot.properties(title="").encode(alt.Y(title="Sediment Yield (ton/yr)")) | y_label_none(model_2_scatterplot.properties(title="")) | y_label_none(model_3_scatterplot.properties(title="")) | y_label_none(model_4_scatterplot.properties(title="")) | y_label_none(model_5_scatterplot.properties(title="")))
    &
    (darea_normalized.encode(alt.Y(title="Specific Sediment Yield (ton/km²/yr)")) | y_label_none(channelslope_normalized) | y_label_none(hillslope_normalized) | y_label_none(glacialretreat_normalized) | y_label_none(lithology_normalized))
).configure_legend(
    titleFontSize=22, labelFontSize=22, orient='top'
).configure_axis(
    labelFontSize=16, titleFontSize=16
).configure_title(
    fontSize=22
)

alt.VConcatChart(...)

# Save results to CSV

In [329]:
net_measurements.to_csv("outputs/power_law_data_new.csv")